In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
%pip install dask dask-expr

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import dask.dataframe as dd

In [6]:
train = dd.read_csv("/home/owner/Documents/DEV/Graphs/Project/graphs_and_structural_analysis/Santander_Product_Recommendation/train_ver2.csv",
                        dtype={'cod_prov': 'float64',
                                    'indrel_1mes': 'object',
                                    'ult_fec_cli_1t': 'object',
                                    'conyuemp': 'object',
                                    'age': 'object',
                                    'age': 'object',
                                    'antiguedad': 'object',
                                    'ind_actividad_cliente': 'float64',
                                    'ind_nom_pens_ult1': 'float64',
                                    'ind_nomina_ult1': 'float64',
                                    'ind_nuevo': 'float64',
                                    'indrel': 'float64',
                                    'tipodom': 'float64'
                                },
            )

In [7]:
train = train.compute()

/home/owner/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (12,16,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [8]:
col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index", "pais_residencia":"residence",
            "sexo":"sex","fecha_alta":"first_date","ind_nuevo":"new_cust","antiguedad":"seniority",
            "indrel":"is_primary","ult_fec_cli_1t":"last_primary_date","indrel_1mes":"cust_type",
            "tiprel_1mes":"cust_rel_type","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel","cod_prov":"province","nomprov":"province_name",
            "ind_actividad_cliente":"active_index","renta":"income","segmento":"segment"
            }

train.rename(col_names, axis = 1, inplace = True)

In [9]:
train.age = pd.to_numeric(train.age, errors='coerce')
train.income = pd.to_numeric(train.income, errors='coerce')
train.seniority = pd.to_numeric(train.seniority, errors='coerce')
train.first_date = pd.to_datetime(train.first_date, errors = 'coerce')
train.cust_type = pd.to_numeric(train.cust_type, errors='coerce')
train['fecha_dato'] = pd.to_datetime(train['fecha_dato'])

In [11]:
train.isna().sum() / len(train)

fecha_dato           0.000000
cust_id              0.000000
emp_index            0.002032
residence            0.002032
sex                  0.002037
age                  0.002032
first_date           0.002032
new_cust             0.002032
seniority            0.002032
is_primary           0.002032
last_primary_date    0.998183
cust_type            0.011039
cust_rel_type        0.010975
residence_index      0.002032
foreigner_index      0.002032
spouse_index         0.999868
channel              0.013638
indfall              0.002032
tipodom              0.002032
province             0.006858
province_name        0.006858
active_index         0.002032
income               0.204756
segment              0.013876
ind_ahor_fin_ult1    0.000000
ind_aval_fin_ult1    0.000000
ind_cco_fin_ult1     0.000000
ind_cder_fin_ult1    0.000000
ind_cno_fin_ult1     0.000000
ind_ctju_fin_ult1    0.000000
ind_ctma_fin_ult1    0.000000
ind_ctop_fin_ult1    0.000000
ind_ctpp_fin_ult1    0.000000
ind_deco_f

In [12]:
# For features with missing values percentage < 10%, use the most common factor level to impute
cols = ['emp_index','residence','sex','first_date','new_cust','is_primary',"cust_type","cust_rel_type",
       "province","province_name","active_index","channel","segment"]

for i in cols:
    train.loc[train[i].isnull(), i] = train[i].value_counts().index[0]
    
# For features with missing value accounts for over 10%, impute their missing values based on the mean at different dimensions
# For Income, impute the missings using the medians of Customer Segment and Province
train['income'].fillna(train['income'].mean(), inplace = True)


/tmp/ipykernel_5217/9000709.py:10: ChainedAssignmentError: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
When using the Copy-on-Write mode, such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object.


  train['income'].fillna(train['income'].mean(), inplace = True)


In [13]:
# Drop the last primary date and spouse index fields given over 99% missing values
train.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)

# Энжиниринг фич

In [14]:
# 1) Age group: group the age for lifestages: young, adult, senior, elder
age_group = [0, 20, 45, 65, 100]
age_labels = ['young', 'adult', 'senior', 'elder']

train['age_grouped'] = pd.cut(train.age, bins = age_group, labels = age_labels)

In [15]:
# 2) Income Group: group the incomes at Low, Ordinary, Median-high and High classes at (real case may be more complex)
import numpy as np
df_income_group = [0, np.nanpercentile(train['income'], 25), np.nanpercentile(train['income'], 50), np.nanpercentile(train['income'], 75), np.nanpercentile(train['income'], 100)]
income_labels = ['Low','Ordinary','Median-high','High']

train['income_grouped'] = pd.cut(train['income'], df_income_group, labels = income_labels)

In [16]:
categorical = train.columns[(train.dtypes == "object") | (train.dtypes == "category")]

In [17]:
for column in categorical:
    train = pd.concat([train, pd.get_dummies(train[column], prefix=column)], axis=1)
    train.drop(columns=column, inplace=True)

# A теперь положим в нормальный файл

In [18]:
%pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
train.to_feather("train.feather")